In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import time

scope = "user-library-read"
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

def find_songs_genre(genre, n):
    results = sp.recommendations(seed_genres=[genre], limit=n)
    songs = {}
    for track in results['tracks']:
        attr = {}
        auido_feat = sp.audio_features(track["id"])
        attr["name"] = track["name"]
        attr["artists"] = track["artists"][0]["name"]
        attr["length"] = round(track["duration_ms"]/1000)
        attr["pop"] = track["popularity"]
        attr["valence"] = round(auido_feat[0]["valence"]*100, 2)
        attr["energy"] = round(auido_feat[0]["energy"]*100, 2)
        songs[track["name"]] = attr
    return songs


def make_a_list(genre, n):
    songs = {}
    while len(songs) < 100:
        songs_add = find_songs_genre(genre, n)
        songs.update(songs_add)
        time.sleep(1)
    return songs

def create_json(genre, n):
    with open(f'song_db/{genre}.json', 'w') as f:
        json.dump(make_a_list(genre, n), f, indent=4, sort_keys=True)

In [ ]:
def for_every_genre(file, n):
    with open(file, "r") as f:
        genres = [line.strip() for line in f.readlines()]
        genres = [line.strip("\'") for line in genres]
    for genre in genres:
        create_json(genre, n)

file_name = "genres_remain.txt"
for_every_genre(file_name, 100)